# Expolore LUAD vs LUSC 1vsAll

Andrew E. Davidson
aedavids@ucsc.edu
1/2/24


ref: extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/hyperparameterTunningResults1.ipynb  

## Abstract
hyperparameterTunningResults1 did not work as well as we would like for elife clases. see hyperparameterTunningResults1.ipynb for details and oppurtnities to improve
```
best100Enriched_6_Degree1GTEx_TCGA
mean_sensitivity	std_sensitivity	numGenes	numTypes	numDegree1	>0.7	Lung	LUAD	LUSC
0.798024	        0.187475	    329	         83	        76	         62	    0.994	0.673	0.704
```

When we explore the classification errors. It appears that we may be able to improve LUAD and LUSC sensitivity by adding a few discriminating genes
```
private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best100Enriched_6_Degree1GTEx_TCGA/training/best100Enriched_6_Degree1GTEx_TCGA.sh.out/lungExploreClassificationErrors.sh.out

$ cat LUAD.falseNegativeGroupByCounts.csv | grep LUSC
LUSC,25

$ cat LUAD.falsePositiveGroupByCounts.csv | grep LUSC
LUSC,6

$ cat LUSC.falseNegativeGroupByCounts.csv | grep LUAD
LUAD,6

$ cat LUSC.falsePositiveGroupByCounts.csv | grep LUAD
LUAD,25
```

Our 1 vs. all differential expression identified 1,749 out of 74,777 gene loci that meet our criteria for biologic signifigance. 29 of these where found in best100Enriched_6_Degree1GTEx_TCGA.sh.out/upsetPlot.out/best100_degree1.intersection.dict. Only 1 gene in our top 20 LUAD vs. LUSC results was found in the intersection.dict

## <span style="color:red">TODO</span> 
hard code a couple of our top genes into best100Enriched_6_Degree1_selectiveEnrich_LUAD_LUSC_* . Use hyperparameterTunningResults1.ipynb to see if sensitivity improves

Try running LUAD vs Lung and LUSC vs Lung  

There are no falsePostivies of type Lung  
best200GTEx_TCGA/training/best200GTEx_TCGA.sh.out/lungExploreClassificationErrors.sh.out  
```
$ cat LUAD.falseNegativeGroupByCounts.csv |grep Lung
Lung,20
$ cat LUAD.falsePositiveGroupByCounts.csv |grep Lung

$ cat LUSC.falseNegativeGroupByCounts.csv |grep Lung
Lung,12
$ cat LUSC.falsePositiveGroupByCounts.csv |grep Lung
```

check for LUAD/LUSC classification error
```
$ cat LUAD.falseNegativeGroupByCounts.csv |grep LUSC
LUSC,10
$ cat LUAD.falsePositiveGroupByCounts.csv |grep LUSC
LUSC,3

$ cat LUSC.falseNegativeGroupByCounts.csv |grep LUAD
LUAD,3
cat LUSC.falsePositiveGroupByCounts.csv |grep LUAD
LUAD,10
```


In [1]:
import ipynbname

# use display() to print an html version of a data frame
# useful if dataFrame output is not generated by last like of cell
from IPython.display import display
import logging

 # we only configure logging in main module
#loglevel = "WARN"
loglevel = "INFO"
# logFMT = "%(asctime)s %(levelname)s [thr:%(threadName)s %(name)s %(funcName)s() line:%(lineno)s] [%(message)s]"
logFMT = "%(asctime)s %(levelname)s %(name)s %(funcName)s() line:%(lineno)s] [%(message)s]"
logging.basicConfig(format=logFMT, level=loglevel)    

#logger = logging.getLogger(os.path.basename(__file__))

import numpy as np
import pandas as pd
# display all columns
pd.set_option('display.max_columns', None)

import pathlib as pl
import pprint as pp
import os
import sys

2024-01-03 10:40:54,841 INFO numexpr.utils _init_num_threads() line:145] [Note: detected 160 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.]
2024-01-03 10:40:54,842 INFO numexpr.utils _init_num_threads() line:148] [Note: NumExpr detected 160 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
2024-01-03 10:40:54,843 INFO numexpr.utils _init_num_threads() line:160] [NumExpr defaulting to 8 threads.]


In [2]:
# setting the python path allows us to run python scripts from using
# the CLI. 
ORIG_PYTHONPATH = os.environ['PYTHONPATH']

notebookPath = ipynbname.path()
deconvolutionModules = notebookPath.parent.joinpath("../python")
print("deconvolutionModules: {}\n".format(deconvolutionModules))

PYTHONPATH = ORIG_PYTHONPATH + f':{deconvolutionModules}'
print("PYTHONPATH: {}\n".format(PYTHONPATH))

os.environ["PYTHONPATH"] = PYTHONPATH
PYTHONPATH = os.environ["PYTHONPATH"]
print("PYTHONPATH: {}\n".format(PYTHONPATH))

deconvolutionModules: /private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/tunning/../python

PYTHONPATH: :/private/home/aedavids/extraCellularRNA/src:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/tunning/../python

PYTHONPATH: :/private/home/aedavids/extraCellularRNA/src:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/tunning/../python



In [3]:
# to be able to import our local python files we need to set the sys.path
# https://stackoverflow.com/a/50155834
sys.path.append( str(deconvolutionModules) )
print("\nsys.path:\n{}\n".format(sys.path))


sys.path:
['/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/tunning', '/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/tunning', '/private/home/aedavids/extraCellularRNA/src', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python311.zip', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/lib-dynload', '', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/site-packages', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/site-packages/setuptools-57.4.0-py3.9.egg', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/site-packages/pip-21.1.3-py3.9.egg', '/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/tunning/../python']



In [4]:
# to be able to import our local python files we need to set the sys.path
# https://stackoverflow.com/a/50155834
sys.path.append( str(deconvolutionModules) )
print("\nsys.path:\n{}\n".format(sys.path))


sys.path:
['/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/tunning', '/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/tunning', '/private/home/aedavids/extraCellularRNA/src', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python311.zip', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/lib-dynload', '', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/site-packages', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/site-packages/setuptools-57.4.0-py3.9.egg', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/site-packages/pip-21.1.3-py3.9.egg', '/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/tunning/../python', '/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/tunning/../python']



In [5]:
from analysis.bestSignatureGeneConfig import BestSignatureGeneConfig
from analysis.hyperParameterTunningMetrics import elifeCols, lungCols, findFile, findSummaryMetricsCols
from analysis.hyperParameterTunningMetrics import metricsRunner

root = "/private/groups/kimlab/aedavids/deconvolution/LUAD.vs.LUSC"
notebookName = ipynbname.name()
outDir = f'{root}/{notebookName}.out'
print( f'output dir: \n{outDir}' )
os.makedirs(outDir, exist_ok=True)

# reset. remove old cached files
localCacheRootPath = f"{outDir}/{notebookName}.tmp"
! echo rm -rf $localCacheRootPath
os.makedirs(localCacheRootPath, exist_ok=True)

output dir: 
/private/groups/kimlab/aedavids/deconvolution/LUAD.vs.LUSC/expoloreLUAD.vs.LUSC_1vsAll.out
rm -rf /private/groups/kimlab/aedavids/deconvolution/LUAD.vs.LUSC/expoloreLUAD.vs.LUSC_1vsAll.out/expoloreLUAD.vs.LUSC_1vsAll.tmp


In [6]:
skipNRows = 7
resultsFile = "LUAD_vs_all.results"
resultsFilePath = f'{root}/{resultsFile}'
df = pd.read_csv( resultsFilePath, header=skipNRows )

print(f'df.shape: {df.shape}' )
df.head()

df.shape: (74777, 7)


,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,S1PR5,326.362299,-4.042770,0.109002,-37.088837,4.251968e-301,1.530326e-296
1,NTRK2,9792.192393,-6.158155,0.168329,-36.584033,5.132305e-293,9.235840e-289
2,ADAM23,3010.356812,-5.534011,0.152363,-36.321222,7.482562e-289,8.976829e-285
3,EDDM13,79.192781,-5.654201,0.158033,-35.778617,2.375432e-280,2.137354e-276
4,TP63,13736.701694,-5.360654,0.151476,-35.389414,2.484291e-274,1.788243e-270


In [7]:
topN = 10
bsgc = BestSignatureGeneConfig(
    dataSetName="LUAD_LUSC", 
    design="tilda_gender_category", 
    padjThreshold=0.001, 
    lfcThreshold=2.0, 
    n=topN, 
    localCacheRootPath=localCacheRootPath, 
    title="LUAD.vs.LUSC",
)

2024-01-03 10:40:55,431 INFO analysis.bestSignatureGeneConfig __init__() line:59] [BEGIN]
2024-01-03 10:40:55,432 INFO analysis.bestSignatureGeneConfig __init__() line:60] [END]


In [8]:
sortedDF = bsgc._select(df, resultsFile)
print(f'sortedDF.shape : {sortedDF.shape}')

top20Genes = sortedDF.head(n=20).loc[:, "name"].values
print(f'top20Genes \n {top20Genes}')
sortedDF.head(n=20)


2024-01-03 10:40:55,439 INFO analysis.bestSignatureGeneConfig _select() line:99] [BEGIN]
2024-01-03 10:40:55,441 INFO analysis.bestSignatureGeneConfig _select() line:107] [number of genes with padj < 0.001 : 17764]
2024-01-03 10:40:55,446 INFO analysis.bestSignatureGeneConfig _select() line:123] [END]


sortedDF.shape : (1749, 8)
top20Genes 
 ['SFTPB' 'KRT6A' 'KRT17' 'SLC34A2' 'CEACAM6' 'PGC' 'NAPSA' 'PERP' 'GPNMB'
 'S100A9' 'SERPINA1' 'MUC1' 'SLC2A1' 'CEACAM5' 'PIGR' 'MUC5B' 'KRT15'
 'TFRC' 'SFN' 'PKP1']


,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,absLog2FoldChange
1175,SFTPB,293628.132946,2.903266,0.180764,16.061042,4.784464e-58,1.464266e-56,2.903266
252,KRT6A,128767.411290,-4.959188,0.221937,-22.345061,1.348646e-110,1.918542e-108,4.959188
132,KRT17,85428.277489,-4.361951,0.175904,-24.797272,9.593445e-136,2.596073e-133,4.361951
619,SLC34A2,56810.835725,2.858376,0.153681,18.599444,3.246554e-77,1.884625e-75,2.858376
366,CEACAM6,48825.537402,3.195449,0.152621,20.937218,2.453754e-97,2.406350e-95,3.195449
73,PGC,48630.563927,6.778715,0.256681,26.409062,1.078213e-153,5.174127e-151,6.778715
351,NAPSA,48620.496474,3.549034,0.168299,21.087672,1.032174e-98,1.055368e-96,3.549034
18,PERP,43868.584388,-2.400704,0.078110,-30.734814,1.951238e-207,3.696158e-204,2.400704
453,GPNMB,43002.638572,-2.260558,0.113954,-19.837399,1.415987e-87,1.122528e-85,2.260558
3024,S100A9,39984.982917,-2.127904,0.177427,-11.993124,3.860666e-33,4.593363e-32,2.127904


In [9]:
# are any of these in best100Enriched_6_Degree1GTEx_TCGA	?

import ast
from analysis.utilities import findAllGenes
geneNames = sortedDF.loc[:, "name"].values
print(f'len(geneNames) : {len(geneNames)} ' )

runName = "best100Enriched_6_Degree1GTEx_TCGA"
pipelineOutRoot = "/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category"
pp2 = f'{pipelineOutRoot}/{runName}/training/best100Enriched_6_Degree1GTEx_TCGA.sh.out'
intersectionDictPath = f'{pp2}/upsetPlot.out/best100_degree1.intersection.dict'
print(f'intersectionDictPath:\n{intersectionDictPath}')

with open(intersectionDictPath) as f: 
            data = f.read() 

intersectionDict = ast.literal_eval(data)

allGenesSet = findAllGenes(intersectionDict)

i = 0
foundGenesSet = set()
for geneName in geneNames:
    if geneName in allGenesSet:
        print( f' i : {i} found {geneName}' )
        foundGenesSet.add( geneName )

    i += 1

len(geneNames) : 1749 
intersectionDictPath:
/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best100Enriched_6_Degree1GTEx_TCGA/training/best100Enriched_6_Degree1GTEx_TCGA.sh.out/upsetPlot.out/best100_degree1.intersection.dict
 i : 5 found PGC
 i : 37 found ATP1B3
 i : 46 found ABCC5
 i : 47 found IRF6
 i : 65 found CPM
 i : 71 found SERPINB5
 i : 79 found SPRR1B
 i : 83 found GPC3
 i : 88 found FAT2
 i : 104 found LY6D
 i : 112 found DDAH1
 i : 118 found CYP4B1
 i : 124 found LMO3
 i : 127 found GJB6
 i : 136 found PITX1
 i : 150 found LGALS7B
 i : 156 found TSPAN8
 i : 177 found OLFM1
 i : 178 found PLEKHA6
 i : 179 found HPN
 i : 198 found ALPK3
 i : 202 found HR
 i : 238 found COL4A6
 i : 267 found F5
 i : 472 found POMC
 i : 569 found FLG
 i : 598 found GAP43
 i : 845 found MOBP
 i : 1089 found FLG2


In [10]:
# are these good?
selectRows = sortedDF.loc[:, "name"].isin(foundGenesSet)
print(sum(selectRows))
aedwipDF = sortedDF.loc[selectRows, :]
print(f'aedwipDF.shape : {aedwipDF.shape}')
aedwipDF

29
aedwipDF.shape : (29, 8)


,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,absLog2FoldChange
73,PGC,48630.563927,6.778715,0.256681,26.409062,1.078213e-153,5.174127e-151,6.778715
10,ATP1B3,12842.369332,-2.345845,0.071850,-32.649004,8.278312e-234,2.708588e-230,2.345845
17,ABCC5,10017.391568,-2.850095,0.092304,-30.877292,2.410550e-209,4.819894e-206,2.850095
41,IRF6,9869.694650,-2.058809,0.071679,-28.722542,1.995716e-181,1.710186e-178,2.058809
401,CPM,6407.105783,2.330783,0.113750,20.490356,2.624660e-93,2.349854e-91,2.330783
612,SERPINB5,5702.460749,-3.456798,0.185530,-18.632025,1.767090e-77,1.037510e-75,3.456798
677,SPRR1B,4951.914967,-4.533538,0.248344,-18.255077,1.885459e-74,1.000878e-72,4.533538
1407,GPC3,4745.023448,-2.301402,0.150962,-15.244952,1.778626e-52,4.546487e-51,2.301402
14,FAT2,4385.358555,-5.232596,0.168073,-31.132910,8.640858e-213,2.073288e-209,5.232596
1706,LY6D,3734.934914,-3.463505,0.239865,-14.439384,2.924428e-47,6.165969e-46,3.463505


In [11]:
# where top 20 found?
set(top20Genes).intersection(foundGenesSet)


{'PGC'}